# Sample API Call

In [6]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from config import API_KEY

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'25',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': API_KEY,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'timestamp': '2025-02-02T20:19:19.869Z', 'error_code': 0, 'error_message': None, 'elapsed': 18, 'credit_count': 1, 'notice': None, 'total_count': 10808}, 'data': [{'id': 1, 'name': 'Bitcoin', 'symbol': 'BTC', 'slug': 'bitcoin', 'num_market_pairs': 11888, 'date_added': '2010-07-13T00:00:00.000Z', 'tags': ['mineable', 'pow', 'sha-256', 'store-of-value', 'state-channel', 'coinbase-ventures-portfolio', 'three-arrows-capital-portfolio', 'polychain-capital-portfolio', 'binance-labs-portfolio', 'blockchain-capital-portfolio', 'boostvc-portfolio', 'cms-holdings-portfolio', 'dcg-portfolio', 'dragonfly-capital-portfolio', 'electric-capital-portfolio', 'fabric-ventures-portfolio', 'framework-ventures-portfolio', 'galaxy-digital-portfolio', 'huobi-capital-portfolio', 'alameda-research-portfolio', 'a16z-portfolio', '1confirmation-portfolio', 'winklevoss-capital-portfolio', 'usv-portfolio', 'placeholder-ventures-portfolio', 'pantera-capital-portfolio', 'multicoin-capital-portfolio', 'par

In [7]:
type(data)

dict

In [8]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [10]:
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')

In [12]:
df.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,11888,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.981882e+07,1.981882e+07,False,NaN,1,NaN,NaN,None,2025-02-02T20:18:00.000Z,98430.225991,5.705906e+10,100.3548,1.358299,-2.963287,-6.293808,-0.118567,-0.167239,45.446189,1.950770e+12,59.2815,2.067035e+12,None,2025-02-02T20:18:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-03 00:19:41.470600
1,1027,Ethereum,ETH,ethereum,9908,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.205231e+08,1.205231e+08,True,NaN,2,NaN,NaN,None,2025-02-02T20:18:00.000Z,2977.520287,3.602715e+10,86.2822,2.477897,-6.434365,-10.761867,-17.453734,-23.139631,22.806600,3.588599e+11,10.9141,3.588599e+11,None,2025-02-02T20:18:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-03 00:19:41.470600
2,52,XRP,XRP,xrp,1520,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,5.770373e+10,9.998654e+10,False,NaN,3,NaN,NaN,None,2025-02-02T20:18:00.000Z,2.704190,1.339962e+10,236.5127,5.730796,-7.688874,-13.505998,9.958349,11.336251,433.525405,1.560419e+11,4.7419,2.704190e+11,None,2025-02-02T20:18:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-03 00:19:41.470600
3,825,Tether USDt,USDT,tether,115327,2015-02-25T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, ethereum...",NaN,1.394042e+11,1.425729e+11,True,NaN,4,NaN,NaN,None,2025-02-02T20:18:00.000Z,0.999774,1.324794e+11,88.6978,0.057940,-0.007152,-0.000862,0.008069,-0.160334,0.040392,1.393727e+11,4.2388,1.425406e+11,None,2025-02-02T20:18:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2025-02-03 00:19:41.470600
4,5426,Solana,SOL,solana,835,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,4.869806e+08,5.932640e+08,True,NaN,5,NaN,NaN,None,2025-02-02T20:18:00.000Z,205.499578,7.380594e+09,140.7429,4.649802,-6.533070,-19.566530,-5.101190,-12.356746,29.024262,1.000743e+11,3.0411,1.219155e+11,None,2025-02-02T20:18:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-03 00:19:41.470600


# Automated Data Pull

In [7]:
import os
import pandas as pd
from time import time
from time import sleep
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from config import API_KEY

In [8]:
def api_call(url, parameters,headers):
  session = Session()
  session.headers.update(headers)
  try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    return data
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)

In [9]:
def extract_data(data):
  df = pd.json_normalize(data['data'])
  df['timestamp'] = pd.to_datetime('now')
  return df

In [10]:
def time_series_call(url, parameters,headers, iterations, sleep_time):
    df_list = []  
    for _ in range(iterations):
        data = api_call(url, parameters,headers)  
        extracted_df = extract_data(data) 
        df_list.append(extracted_df) 
        print('Data extracted at:', pd.to_datetime('now'))
        sleep(sleep_time)

    df = pd.concat(df_list, ignore_index=True)
    return df


In [11]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'25',
  'convert':'USD'
}

headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': API_KEY,
}

In [12]:
crypto_data = time_series_call(url, parameters,headers, 5, 10)

Data extracted at: 2025-02-04 23:31:43.295463
Data extracted at: 2025-02-04 23:31:54.234650
Data extracted at: 2025-02-04 23:32:05.305487
Data extracted at: 2025-02-04 23:32:16.262807
Data extracted at: 2025-02-04 23:32:27.614493


In [13]:
crypto_data.tail()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
120,11092,Bitget Token,BGB,bitget-token-new,29,2021-07-29T00:00:00.000Z,"[centralized-exchange, ethereum-ecosystem]",NaN,1.200000e+09,1.200000e+09,...,0.2372,7.868153e+09,None,2025-02-04T19:31:00.000Z,1027.0,Ethereum,ETH,ethereum,0x54D2252757e1672EEaD234D27B1270728fF90581,2025-02-04 23:32:27.613019
121,6636,Polkadot,DOT,polkadot-new,864,2020-08-19T00:00:00.000Z,"[substrate, polkadot, polkadot-ecosystem, thre...",NaN,1.545524e+09,1.545524e+09,...,0.2295,7.610733e+09,None,2025-02-04T19:31:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-04 23:32:27.613019
122,1831,Bitcoin Cash,BCH,bitcoin-cash,975,2017-07-23T00:00:00.000Z,"[mineable, pow, sha-256, marketplace, medium-o...",21000000.0,1.982478e+07,1.982478e+07,...,0.2056,7.223895e+09,None,2025-02-04T19:31:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-04 23:32:27.613019
123,29470,Ethena USDe,USDe,ethena-usde,102,2024-02-20T11:05:01.000Z,"[stablecoin, asset-backed-stablecoin, ethereum...",NaN,6.040416e+09,6.040416e+09,...,0.1821,6.039132e+09,None,2025-02-04T19:31:00.000Z,1027.0,Ethereum,ETH,ethereum,0x4c9edd5852cd905f086c759e8383e09bff1e68b3,2025-02-04 23:32:27.613019
124,6536,MANTRA,OM,mantra,240,2020-08-11T00:00:00.000Z,"[cosmos-ecosystem, defi, dao, ethereum-ecosyst...",NaN,9.709641e+08,1.806488e+09,...,0.1758,1.085109e+10,None,2025-02-04T19:31:00.000Z,1027.0,Ethereum,ETH,ethereum,0x3593d125a4f7849a1b059e64f4517a86dd60c95d,2025-02-04 23:32:27.613019


# Conversion to CSV file

In [14]:
if not os.path.exists('crypto_data.csv'):
    crypto_data.to_csv('crypto_data.csv', index=False)